In [1]:
import numpy as np
import pandas as pd 
#import matplotlib.pyplot as plt
#import seaborn as sns
pd.set_option('max_columns', 50)
####model libraries ####################
from sklearn.linear_model import Ridge, Lasso,SGDRegressor,LinearRegression
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
#########################################
from sklearn.cross_decomposition import PLSRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score
from sklearn.preprocessing import MinMaxScaler

#### visulization #####
import plotly.graph_objects as go


READING DATA, INITIALIZATION

In [2]:
whole_data = pd.read_csv('C:/Users/halil/Desktop/forecast_co2/datasets/ethylene_CO.txt', delimiter='\s+',names=[
   "Time",
   "CO2_con",
   "Ethy_con",
   "sensor1",
   "sensor2",
   "sensor3",
   "sensor4",
   "sensor5",
   "sensor6",
   "sensor7",
   "sensor8",
   "sensor9",
   "sensor10",
   "sensor11",
   "sensor12",
   "sensor13",
   "sensor14",
   "sensor15",
   "sensor16",])

#normalization in range (-1,1)

In [3]:

def normalize(df,column_list):
    for col in column_list:
        feature_range = (0,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

function that create train test data and labels from raw dataset

In [4]:

def make_data(dataframe):
    train = dataframe[:int(len(dataframe)*0.8)]
    test = dataframe[int(len(dataframe)*0.8):]

    x_train = train.copy()
    x_test = test.copy()

    y_train = x_train.pop('CO2_con')
    y_test = x_test.pop('CO2_con')

    return x_train, x_test, y_train, y_test 

machine learning model function, to get model errors 

In [5]:
def calculate_xgb(x_train,x_test,y_train,y_test):
    model = XGBRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_linear_reg(x_train,x_test,y_train,y_test):

    model = LinearRegression()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

In [6]:
def time_encoding(dataset):    

    time_adjusted = dataset
    index_to_time = pd.Series(dataset.index/100 )
    converted_time_data = pd.to_datetime(index_to_time, unit="s",origin="2022")
    time_adjusted["Time"] = converted_time_data.dt.round("10U")

    conversion = time_adjusted["Time"].astype(str)
    time_adjusted["hour"] = conversion.apply(lambda x:int(x[11:13]))
    time_adjusted["min"] = conversion.apply(lambda x:int(x[14:16]))
    time_adjusted["sec"] = conversion.apply(lambda x:int(x[17:19]))
    time_adjusted["100hz"] = conversion.apply(lambda x:int(x[20:22]))
    encoded = time_adjusted.drop(["Time"],axis=1)

    return encoded


In [7]:


#new_data = df.drop(["Time"],axis=1)
#co2_data = new_data.drop(["Ethy_con"],axis=1)
#eth_data = new_data.drop(["CO2_con"],axis=1)

In [8]:
#orginized_time = df.copy()
#orginized_time["Time"] = pd.to_datetime(orginized_time["Time"], unit="ms",origin="2022")
#orginized_time['Time'] = orginized_time.Time.dt.round("10U")
#time_series = df[["Time","sensor1"]]
#df = df.set_index('Time')

In [9]:
df= whole_data.copy()
onehot_encoded = time_encoding(df)

In [11]:
onehot_encoded

,CO2_con,Ethy_con,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,sensor7,sensor8,sensor9,sensor10,sensor11,sensor12,sensor13,sensor14,sensor15,sensor16,hour,min,sec,100hz
0,0.0,0.0,-50.85,-1.95,-41.82,1.30,-4.07,-28.73,-13.49,-3.25,55139.95,50669.50,9626.26,9762.62,24544.02,21420.68,7650.61,6928.42,0,0,0,0
1,0.0,0.0,-49.40,-5.53,-42.78,0.49,3.58,-34.55,-9.59,5.37,54395.77,50046.91,9433.20,9591.21,24137.13,20930.33,7498.79,6800.66,0,0,0,1
2,0.0,0.0,-40.04,-16.09,-27.59,0.00,-7.16,-42.14,-12.52,-5.86,53960.02,49299.30,9324.40,9449.81,23628.90,20504.94,7369.67,6697.47,0,0,0,2
3,0.0,0.0,-47.14,-10.57,-32.28,4.40,-11.22,-37.94,-7.16,-1.14,53047.71,48907.00,9170.64,9305.58,23101.66,20101.42,7285.13,6578.52,0,0,0,3
4,0.0,0.0,-33.58,-20.79,-33.25,6.03,3.42,-34.22,-14.46,8.31,52700.28,48330.96,9073.64,9163.47,22689.54,19694.07,7156.74,6468.32,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4208256,0.0,0.0,1127.81,105.93,2117.39,2247.58,1207.59,1646.73,2089.97,2262.98,1061.70,1070.47,2570.33,2073.57,709.06,775.36,2513.75,1848.65,11,41,22,56
4208257,0.0,0.0,1140.73,96.46,2105.93,2253.69,1216.39,1624.47,2073.57,2246.61,1063.89,1075.26,2580.88,2083.78,698.24,768.93,2511.20,1843.63,11,41,22,57
4208258,0.0,0.0,1123.99,107.92,2111.18,2245.63,1215.36,1628.44,2085.45,2271.80,1059.31,1063.49,2583.20,2090.20,709.62,775.73,2510.18,1841.80,11,41,22,58
4208259,0.0,0.0,1131.44,111.08,2114.76,2249.54,1197.99,1612.17,2078.32,2272.29,1063.69,1066.08,2578.56,2100.92,716.72,779.14,2508.91,1851.17,11,41,22,59
